In [92]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr/')
print(os.getcwd())
from pm4py.objects.conversion.dcr import converter
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.utils.dcr_utils import nested_groups_and_sps_to_flat_dcr
from pm4py.objects.conversion.dcr import converter as dcr_to_tapn

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/vco/Projects/pm4py-dcr
/home/vco/Projects/pm4py-dcr


In [110]:
rail_example = 'models/rail_example.xml'
rail_dcr_labeled = dcr_importer.apply(rail_example,parameters={'as_dcr_object':True,'labels_as_ids':True})

In [111]:
nested_groups_and_sps_to_flat_dcr(rail_dcr_labeled)

In [113]:
dcr_dict = rail_dcr_labeled.obj_to_template()

In [117]:
tapn, m = dcr_to_tapn.apply(dcr_dict,variant=dcr_to_tapn.Variants.TO_TIMED_ARC_PETRI_NET)

[] {}
{}


KeyError: executed_approach1